# Visualizing Active Deaths

Look at the past week to see which state is doing the worst right now.

In [ ]:
import numpy as np
import pandas as pd
from importlib import reload
from datetime import datetime, timezone, timedelta
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy import optimize
import statsmodels.api as sm
import os
import pickle
import requests

# Use parent directory as a base code directory.
import sys
sys.path.append("../")

from modeling import dataproc, optimizer, sir_model

# Load Covid-19 and Census Data

In [ ]:
datastore = dataproc.DataStore()

In [ ]:
fips_df = pd.read_csv('../data_repo/state_fips_master.csv')
fips_df.head()

In [ ]:
DAYS_AGO = -270
WINDOW_SIZE = 30
deaths_per_mil = []
for fips in range(1,57):
    try:
        area_df, population = datastore.get_time_series_for_area('US', [(fips, [])])
        state = fips_df[fips_df['fips'] == fips]['state_name'].iloc[0]
        two_week_deaths = ((area_df.iloc[DAYS_AGO-WINDOW_SIZE:] - area_df.iloc[DAYS_AGO-WINDOW_SIZE:].shift(WINDOW_SIZE))['Deaths']
                           / population * 1000000)[WINDOW_SIZE:].to_numpy()
        deaths_per_mil.append((state, two_week_deaths, ))
    except:
        print('failed for fips', fips)
        continue

In [ ]:
fips_df[fips_df['fips'] == 56]['state_name'].iloc[0]

In [ ]:
prev_y_max = 1000
for days in range(DAYS_AGO, 0):
    d = datetime.date(datetime.now()) + timedelta(days)
    plt.figure(figsize=(15, 5))
    plt.title('Date: {}'.format(d))
    plt.bar([x[0] for x in deaths_per_mil], [x[1][days] for x in deaths_per_mil])
    prev_y_max = 0.9 * prev_y_max + 0.12 * np.max([x[1][days] for x in deaths_per_mil])
    plt.ylim((0, prev_y_max))
    plt.xticks(rotation=90)
    plt.xlabel('State')
    plt.ylabel('Deaths per million in last ' + str(WINDOW_SIZE) + ' days')
    plt.savefig('data/images/active_deaths/pic_{:04d}.png'.format(days - DAYS_AGO))
    plt.show()

In [ ]:
area_df, population = datastore.get_time_series_for_area('US', [(34, [])])
sample_df = area_df[(area_df['Date'] >= '2020-06-20') & (area_df['Date'] <= '2020-06-30')]

case_ratio = area_df[area_df['Date'] == '2020-06-25']['Cases'].iloc[0] / area_df[area_df['Date'] == '2020-06-24']['Cases'].iloc[0]
death_ratio = area_df[area_df['Date'] == '2020-06-25']['Deaths'].iloc[0] / area_df[area_df['Date'] == '2020-06-24']['Deaths'].iloc[0]
print(case_ratio, death_ratio)
# deaths_ratio = area_df[]

# area_df['Updated Deaths'] = np.where(area_df['Date'] < '2020-06-25',
#                                      area_df['Deaths'] / area_df[]
# sample_df


In [ ]:
week_trend = area_df[(area_df['Date'] >= '2020-06-17') & (area_df['Date'] < '2020-06-25')]['Deaths'].to_numpy()
adjustment = area_df[area_df['Date'] == '2020-06-25']['Deaths'].iloc[0] / ((week_trend[-1] - week_trend[0]) / 7 + week_trend[-1])

In [ ]:
adjustment * 

In [ ]:
# Plot deaths per mil

states_of_interest = [
    'New York',
    'New Jersey',
    'Louisiana',
    'Mississippi',
    'Arizona',
    'Michigan',
    'Illinois',
    'Florida',
    'Georgia',
    'Texas',
    'California',
]

DAYS_AGO = -180
for state in states_of_interest:
    try:
        fips = fips_df[fips_df['state_name'] == state]['fips'].iloc[0]
        area_df, population = datastore.get_time_series_for_area('US', [(fips, [])])
        dates = area_df.iloc[DAYS_AGO:]['Date'].to_numpy()
        deaths = area_df.iloc[DAYS_AGO:]['Deaths'].to_numpy() / population * 1000000
        plt.plot(dates, deaths, label=state)
    except:
        print('failed for fips', fips)
        continue

plt.legend()

In [ ]:
area_df.iloc[DAYS_AGO:]